## Tutorial on how to use Parcels on 3D C grids

One of the features of Parcels is that it can directly and natively work with `Field` data discretised on C-grids. These C grids are very popular in OGCMs, so velocity fields outputted by OGCMs are often provided on such grids, except if they have been firstly re-interpolated on a A grid.

More information about C-grid interpolation can be found in [Delandmeter et al., 2019](https://www.geosci-model-dev-discuss.net/gmd-2018-339/).
An example of such a discretisation is the NEMO model, which is one of the models supported in Parcels. A tutorial teaching how to [interpolate 2D data on a NEMO grid](https://nbviewer.jupyter.org/github/OceanParcels/parcels/blob/master/parcels/examples/tutorial_nemo_curvilinear.ipynb) is available within Parcels.

Here, we focus on 3D fields. Basically, it is a straightforward extension of the 2D example, but it is very easy to do a mistake in the setup of the vertical discretisation that would affect the interpolation scheme.

### Preliminary comments

*How to know if your data is discretised on a C grid?* The best way is to read the documentation that comes with the data. Alternatively, an easy check is to assess the coordinates of the U, V and W fields: for an A grid, U, V and W are distributed on the same nodes, such that the coordinates are the same. For a C grid, there is a shift of half a cell between the different variables.

*What about grid indexing?* Since the C-grid variables are not located on the same nodes, there is not one obvious way to define the indexing, i.e. where is `u[k,j,i]` compared to `v[k,j,i]` and `w[k,j,i]`. In Parcels, we use the same notation as in NEMO: see [horizontal indexing](https://www.nemo-ocean.eu/doc/img360.png) and [vertical indexing](https://www.nemo-ocean.eu/doc/img362.png).
It is important that you check if your data is following the same notation. Otherwise, you should re-index your data properly (this can be done within Parcels, there is no need to regenerate new netcdf files).

### How to create a 3D NEMO `dimensions` dictionary?

In the following, we will show how to create the `dimensions` dictionary for 3D NEMO simulations. What you require is a 'mesh_mask' file, which in our case is called `coordinates.nc` but in some other versions of NEMO has a different name. In any case, it will have to contain the variables `glamf`, `gphif` and `depthw`, which are the longitude, latitude and depth of the mesh nodes, respectively. Note that `depthw` is not part of the mesh_mask file, but is in the same file as the w data (`wfiles[0]`).

For the C-grid interpolation in Parcels to work properly, it is important that `U`, `V` and `W` are on the same grid.

The code below is an example of how to create a 3D simulation with particles, starting in the mouth of the river Rhine at 1m depth, and advecting them through the North Sea using the `AdvectionRK4_3D`

In [1]:
from parcels import FieldSet, ParticleSet, JITParticle, AdvectionRK4_3D
from glob import glob
import numpy as np
from datetime import timedelta as delta
from os import path

data_path = 'NemoNorthSeaORCA025-N006_data/'
data_path = '/Users/delandmeter/sources/oceanparcels_website/examples-data/NemoNorthSeaORCA025-N006_data/'
ufiles = sorted(glob(data_path+'ORCA025-N06_200001??d05U.nc'))
vfiles = sorted(glob(data_path+'ORCA025-N06_200001??d05V.nc'))
wfiles = sorted(glob(data_path+'ORCA025-N06_200001??d05W.nc'))
mesh_mask = data_path + 'coordinates.nc'

filenames = {'U': {'lon': mesh_mask, 'lat': mesh_mask, 'depth': wfiles[0], 'data': ufiles},
             'V': {'lon': mesh_mask, 'lat': mesh_mask, 'depth': wfiles[0], 'data': vfiles},
             'W': {'lon': mesh_mask, 'lat': mesh_mask, 'depth': wfiles[0], 'data': wfiles}}

variables = {'U': 'uo',
             'V': 'vo',
             'W': 'wo'}
dimensions = {'U': {'lon': 'glamf', 'lat': 'gphif', 'depth': 'depthw', 'time': 'time_counter'},
              'V': {'lon': 'glamf', 'lat': 'gphif', 'depth': 'depthw', 'time': 'time_counter'},
              'W': {'lon': 'glamf', 'lat': 'gphif', 'depth': 'depthw', 'time': 'time_counter'}}

fieldset = FieldSet.from_nemo(filenames, variables, dimensions)

pset = ParticleSet.from_line(fieldset=fieldset, pclass=JITParticle,
                             size=10,
                             start=(2.96, 51.6),
                             finish=(3.26, 51.8),
                             depth=[1]*10)

kernels = pset.Kernel(AdvectionRK4_3D)
pset.execute(kernels, runtime=delta(days=4), dt=delta(hours=6))

print(pset)


         It will be opened with no decoding. Filling values might be wrongly parsed.
INFO: Compiled JITParticleAdvectionRK4_3D ==> /var/folders/h0/01fvrmn11qb62yjw7v1kn62r0000gq/T/parcels-503/7d42b40fa0f501adf2fbf7449ad1f556.so


P[0](lon=3.349206, lat=51.770679, depth=1.011043, time=345600.000000)
P[1](lon=3.378664, lat=51.789280, depth=1.030178, time=345600.000000)
P[2](lon=3.403621, lat=51.805943, depth=1.081050, time=345600.000000)
P[3](lon=3.430356, lat=51.822876, depth=1.117308, time=345600.000000)
P[4](lon=3.456368, lat=51.839638, depth=1.150787, time=345600.000000)
P[5](lon=3.480758, lat=51.856472, depth=1.189697, time=345600.000000)
P[6](lon=3.507661, lat=51.873787, depth=1.213882, time=345600.000000)
P[7](lon=3.531766, lat=51.891018, depth=1.211578, time=345600.000000)
P[8](lon=3.559770, lat=51.908764, depth=1.192187, time=345600.000000)
P[9](lon=3.588396, lat=51.933567, depth=1.223015, time=345600.000000)
